# SSI Algorithm Demo

Add related components from algotrade library 

In [ ]:
import ujson
import time
import logging
from threading import Thread
from algotrade import redis, ssi_api, handler

logging.basicConfig(filename='logs.log', level=logging.INFO)

Proceed returned data from SSI, we will update open position and total profit if the current position is closed        
If `filledQty` is greater than zero, calculate the profit/loss from this position and sum up with `TOTAL_PROFIT`     
After that, update `OPEN_POSITION` to initialized state -> allow to open new position

In [ ]:
ALL_F1M_PRICE_TICKS = 'ALL_F1M'
OPEN_ATC = False
HANDLED_REQUESTS = []

In [ ]:
def log(text):
    logging.info(text)

In [ ]:
def handle_order_event(msg):
    event_data =  ujson.loads(msg['data'])
    log("event_data['data']: {}".format(event_data))
    data = event_data['data']
    event_type = event_data['type']

    if event_type == 'orderEvent':
        global OPEN_POSITION, CURRENT_ORDER, TOTAL_PROFIT, HANDLED_REQUESTS, CANCELLATION_LOCK
        request_id = data['uniqueID']
        log('-------------BEFORE UPDATE------------')
        log('OPEN_POSITION {}'.format(OPEN_POSITION))
        log('CURRENT_ORDER {}'.format(CURRENT_ORDER))
        log('--------------------------------------')
        log('HANDLED_REQUESTS: {}'.format(HANDLED_REQUESTS))
        if CURRENT_ORDER['request_id'] == request_id:
            CURRENT_ORDER['order_id'] = data['orderID']
            if data['filledQty'] > 0 and request_id not in HANDLED_REQUESTS:
                HANDLED_REQUESTS.append(request_id) # avoid same order events are returned

                # update TOTAL_PROFIT
                filled_price = data['avgPrice']
                TOTAL_PROFIT += calculate_profit(filled_price)
                log('TOTAL_PROFIT {}'.format(TOTAL_PROFIT))

                # update OPEN_POSITION
                OPEN_POSITION['avg_price'] = filled_price
                new_qty = abs(OPEN_POSITION['qty'] - data['filledQty'])
                OPEN_POSITION['qty'] = new_qty
                openned_side = CURRENT_ORDER['side'] if new_qty > 0 else None
                OPEN_POSITION['side'] = openned_side
                
                # update CURRENT_ORDER
                CURRENT_ORDER['status'] = 'FILLED'

                log('----------AFTER FILLED----------------')
                log('OPEN_POSITION {}'.format(OPEN_POSITION))
                log('CURRENT_ORDER {}'.format(CURRENT_ORDER))
                log('--------------------------------------')

            update_redis_db()

In [ ]:
%run data.ipynb
%run db.ipynb
%run config.ipynb

Using redis pub/sub to subscribe ticks

In [ ]:
r = redis.init_redis()
pubsub = r.pubsub()
F1 =  redis.get_key('F1M_CODE')
F1_TICK_CHANNEL = 'HNXDS:{}'.format(F1)
SSI_EVENTS_CHANNEL = 'SSI_{}_EVENTS'.format(ACCOUNT)
pubsub.subscribe(F1_TICK_CHANNEL, SSI_EVENTS_CHANNEL)

In [ ]:
def init_local_db(redis_data):
    if redis_data is not None:
        global CURRENT_ORDER, OPEN_POSITION, TOTAL_PROFIT
        redis_data_value = ujson.loads(redis_data)
        log('INIT LOCAL DB: {}'.format(redis_data_value))
        CURRENT_ORDER = redis_data_value['CURRENT_ORDER']
        OPEN_POSITION = redis_data_value['OPEN_POSITION']
        TOTAL_PROFIT = redis_data_value['TOTAL_PROFIT']

In [ ]:
if CUT_LOSS_THRESHOLD >= 0 or TAKE_PROFIT_THRESHOLD <= 0:
    raise Exception('Please check configuration - cut loss must be less than 0 and take profit must be greater than 0')

if CONSUMER_ID == "" or CONSUMER_SECRET == "" or ACCOUNT == "" or PRIVATE_KEY == "":
    raise Exception('Please check configuration - account configuration can not be empty')
    
if START_TRADING_TIME < '09:00:00' or START_TRADING_TIME > '14:30:00':
    raise Exception('Please check configuration - start trading time must be between 09AM and 02:30PM')
    
ssi_api.init_config(
  CONSUMER_ID,
  CONSUMER_SECRET,
  ACCOUNT,
  PRIVATE_KEY
)
try: 
    token = ssi_api.login_with_pin(OTP)
except:
    raise Exception('Invalid OTP - Please update OTP and try again')

ACCOUNT_KEY = 'SSI_{}'.format(ACCOUNT)
redis_data = redis.get_key(ACCOUNT_KEY)
init_local_db(redis_data)
handler.init_stream(HUB_URI, ACCOUNT, CONSUMER_ID, CONSUMER_SECRET)

In [ ]:
def update_redis_db():
    redis.set_key(ACCOUNT_KEY, ujson.dumps(dict({
        'CURRENT_ORDER': CURRENT_ORDER,
        'OPEN_POSITION': OPEN_POSITION,
        'TOTAL_PROFIT': TOTAL_PROFIT
    })))

Initialize data with redis value (updated by algotrade service)

In [ ]:
all_f1_data = redis.get_key(ALL_F1M_PRICE_TICKS)
if all_f1_data is not None:
    init_ticks(ujson.loads(all_f1_data))

Callback function is used to update `CURRENT_ORDER`

In [ ]:
def update_current_order(data: dict):
    global CURRENT_ORDER
    log('-------------BEFORE CALLBACK------------')
    log('CURRENT_ORDER {}'.format(CURRENT_ORDER))
    log('--------------------------------------')
    CURRENT_ORDER = data
    log('-------------AFTER CALLBACK------------')
    log('CURRENT_ORDER {}'.format(CURRENT_ORDER))
    log('--------------------------------------')
    update_redis_db()

This is wrapper function - used to open new position

In [ ]:
def open_position(side: str, order_type: str, price: float, reverse: bool):
    current_side = OPEN_POSITION['side']
    current_order_status = CURRENT_ORDER['status']
    handler.open_position(
        ACCOUNT,
        F1,
        side,
        order_type,
        price,
        reverse,
        MAX_ROUND,
        current_side,
        current_order_status,
        update_current_order
    )

Because we want to open position with latest price (ASAP), so we need to pass ceiling price if we buy (open LONG), otherwise, pass floor price if we sell (open SHORT)

In [ ]:
def handle_position_with_price(side: str, order_type: str, reverse: bool):
    price = CEILING_PRICE if side == 'BUY' else FLOOR_PRICE
    price = 0 if order_type == 'ATC' else price
    open_position(side, order_type, price, reverse)

In [ ]:
def get_reverse_side(side: str):
    return 'BUY' if side == 'SELL' else 'SELL'

In [ ]:
def get_sma_value(value):
    return 'N/A' if value == 0 else value

### Algorithm implementation
Each tick has `last_px` (last price) field, we will add into price list and calculate SMA(t), SMA(t-1)  
About SMA (Simple Moving Average): https://www.investopedia.com/terms/s/sma.asp   

- If last_px(t-1) < SMA(t-1) and last_px(t) >= SMA(t) -> Open **Long**
- If last_px(t-1) > SMA(t-1) and last_px(t) <= SMA(t) -> Open **Short**

If unrealized profit/loss exceed our range (is configured in config.ipynb), we will close position to take profit or cut loss (reverse position)   
We will close openned position in ATC session if we have

In [ ]:
TRIGGER_ATC_TIME = '14:30:05'
def handle_msg_internal(hidden_info: dict):
    if 'LastPrice' not in hidden_info:
        return
    last_px = hidden_info['LastPrice']
    trade_time = hidden_info['Time']
    global OPEN_POSITION, START_TRADING_TIME, FLOOR_PRICE, CEILING_PRICE
    if last_px is not None:
        [prev_last_px, prev_sma, sma] = add_tick(last_px)
        log('Time: {}, SMA(t-1): {}, SMA(t): {}, LAST_PX(t-1): {}, LAST_PX(t): {}'.format(trade_time, get_sma_value(prev_sma), get_sma_value(sma), prev_last_px, last_px))
        if FLOOR_PRICE is None or CEILING_PRICE is None:
            FLOOR_PRICE = hidden_info['Floor']
            CEILING_PRICE = hidden_info['Ceiling']
                    
        if trade_time >= START_TRADING_TIME and trade_time < TRIGGER_ATC_TIME:
            if OPEN_POSITION['side'] is not None:
                unrealized = calculate_profit(last_px)
                log('Unrealized Profit/Loss: {}'.format(calculate_profit(last_px)))
                if unrealized <= CUT_LOSS_THRESHOLD or unrealized >= TAKE_PROFIT_THRESHOLD:
                    # cut loss or take profit -> close openning position
                    print('Cut loss' if unrealized < 0 else 'Take profit')
                    handle_position_with_price(get_reverse_side(OPEN_POSITION['side']), 'LO', False)
            elif prev_sma > 0.0:
                if prev_last_px < prev_sma and last_px >= sma:
                    log('Long Signal')
                    handle_position_with_price('BUY', 'LO', False)
                if prev_last_px > prev_sma and last_px <= sma:
                    log('Short Signal')
                    handle_position_with_price('SELL', 'LO', False)

    if trade_time >= TRIGGER_ATC_TIME and OPEN_POSITION['side'] is not None:
        # close opening position in ATC session
        log('Close ATC')
        handle_position_with_price(get_reverse_side(OPEN_POSITION['side']), 'ATC', False)

In [ ]:
def handle_msg(msg):
    msg_data = ujson.loads(msg['data'])
    if msg_data['hidden_system_status'] is not None:
        hidden_info = ujson.loads(msg_data['hidden_system_status'])
        handle_msg_internal(hidden_info)
        
        # draw_chart()

In [ ]:
def pub_sub():
    for message in pubsub.listen():
        if message['type'] == 'message':
            channel = message['channel']
            if channel == F1_TICK_CHANNEL:
                handle_msg(message)
            elif channel == SSI_EVENTS_CHANNEL:
                handle_order_event(message)

In [ ]:
redis_thread = Thread(target=pub_sub)
redis_thread.start()

In [ ]:
def tail_log(thefile):
     while True:
        line = thefile.readline()
        if not line or not line.endswith('\n'):
            time.sleep(0.1)
            continue
        yield line

def init_printing_log():
    logfile = open("logs.log", "r")
    loglines = tail_log(logfile)
    for line in loglines:
        print(line, end='')

## If you want to show log realtime -> please stop and re-run the below cell &#8595;

In [ ]:
init_printing_log()